# 依赖

In [4]:
# --- 导入依赖库 ---
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from urllib.parse import urljoin, urlparse
from datetime import datetime
from tqdm.notebook import tqdm
!pip install selenium webdriver-manager
!pip install selenium webdriver_manager pandas
# --- 🔧 用户配置区 (请在此处替换) ---
# 1. 目标大学名称 (用于文件名和日志)
UNIVERSITY_NAME = "HKU"  # 例如: "HKU"

# 2. 只有在此域名下的链接才会被进一步抓取 (防止爬虫跑偏到 Facebook/Youtube 等外链)
ALLOWED_DOMAIN = "hku.hk"    # 例如: "hku.hk"

# 3. 所有的 Master List 入口链接 (可以是多个)
PROGRAM_LIST_URLS = [
    "https://portal.hku.hk/tpg-admissions/programme-listing", 
    # "<<REPLACE_PROGRAM_LIST_URL_2>>",
]

# --- 全局爬虫配置 ---
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
}
TIMEOUT = 15  # 请求超时秒数
MAX_RETRIES = 3 # 最大重试次数

print(f"✅ 配置已加载: 目标大学 THE UNIVERSITY OF HONGKONG] | 允许域名 [https://portal.hku.hk/tpg-admissions/programme-listing]")

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
✅ 配置已加载: 目标大学 THE UNIVERSITY OF HONGKONG] | 允许域名 [https://portal.hku.hk/tpg-admissions/programme-listing]


# 爬虫 12.10 

## Version 1

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML

# --- 计时开始 (总任务) ---
total_start_time = time.time()

# --- 1. 浏览器配置 ---
chrome_options = Options()
# chrome_options.add_argument("--headless")  # 调试期间建议注释掉，开启无头模式可加快一点速度
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

target_url = "https://portal.hku.hk/tpg-admissions/programme-listing"
base_url = "https://portal.hku.hk"

all_valid_data = []

try:
    print(f"正在访问首页: {target_url}")
    driver.get(target_url)
    
    # 等待初始页面加载
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]'))
    )
    
    page_num = 1
    
    while True:
        # --- 计时开始 (当前页) ---
        page_start_time = time.time()
        print(f"\n--- 正在抓取第 {page_num} 页 ---")
        
        # 1. 抓取当前页面的数据
        current_links = driver.find_elements(By.TAG_NAME, "a")
        
        count_in_page = 0
        for link in current_links:
            try:
                url = link.get_attribute("href")
                text = link.text.strip()
                
                if url and "programme-details" in url and text:
                    full_url = urljoin(base_url, url)
                    
                    if not any(d['link'] == full_url for d in all_valid_data):
                        all_valid_data.append({
                            "program_name": text,
                            "link": full_url
                        })
                        count_in_page += 1
            except:
                continue
        
        # --- 计时结束 (当前页) ---
        page_end_time = time.time()
        page_duration = page_end_time - page_start_time
        print(f"✅ 第 {page_num} 页完成。本页新增 {count_in_page} 个项目。耗时: {page_duration:.2f} 秒")

        # 2. 尝试翻页逻辑
        try:
            next_buttons = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            
            if not next_buttons:
                print("未找到下一页按钮，抓取结束。")
                break
            
            next_btn = next_buttons[0]
            
            parent_li = next_btn.find_element(By.XPATH, "./..")
            if "disabled" in parent_li.get_attribute("class"):
                print("已到达最后一页。")
                break
                
            driver.execute_script("arguments[0].click();", next_btn)
            page_num += 1
            
            # 等待加载
            time.sleep(3) 
            
        except Exception as e:
            print(f"翻页停止: {str(e)}")
            break

finally:
    driver.quit()
    
    # --- 计时统计 ---
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    minutes = int(total_duration // 60)
    seconds = int(total_duration % 60)
    
    print("="*40)
    print(f"🎉 全部结束！")
    print(f"📊 总共抓取: {len(all_valid_data)} 个项目")
    print(f"⏱️ 总耗时: {minutes} 分 {seconds} 秒")
    if page_num > 1:
        print(f"⚡ 平均每页耗时: {total_duration / page_num:.2f} 秒")
    print("="*40)

# --- 结果展示 ---
if all_valid_data:
    df = pd.DataFrame(all_valid_data)
    df.to_csv("hku_programs_all_pages.csv", index=False, encoding="utf-8-sig")
    print("文件已保存为 hku_programs_all_pages.csv")
    
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        return f'<a target="_blank" href="{val}">{val}</a>'
    
    print("\n↓↓↓ 结果预览 (点击链接跳转) ↓↓↓")
    display(HTML(df.head(20).style.format({'link': make_clickable}).to_html(escape=False)))
else:
    print("没有抓取到数据。")

正在启动浏览器...
正在访问首页: https://portal.hku.hk/tpg-admissions/programme-listing

--- 正在抓取第 1 页 ---
✅ 第 1 页完成。本页新增 10 个项目。耗时: 0.72 秒

--- 正在抓取第 2 页 ---
✅ 第 2 页完成。本页新增 10 个项目。耗时: 0.72 秒

--- 正在抓取第 3 页 ---
✅ 第 3 页完成。本页新增 10 个项目。耗时: 0.85 秒

--- 正在抓取第 4 页 ---
✅ 第 4 页完成。本页新增 10 个项目。耗时: 0.91 秒

--- 正在抓取第 5 页 ---
✅ 第 5 页完成。本页新增 10 个项目。耗时: 1.18 秒

--- 正在抓取第 6 页 ---
✅ 第 6 页完成。本页新增 10 个项目。耗时: 0.87 秒

--- 正在抓取第 7 页 ---
✅ 第 7 页完成。本页新增 10 个项目。耗时: 0.86 秒

--- 正在抓取第 8 页 ---
✅ 第 8 页完成。本页新增 10 个项目。耗时: 0.85 秒

--- 正在抓取第 9 页 ---
✅ 第 9 页完成。本页新增 10 个项目。耗时: 0.83 秒

--- 正在抓取第 10 页 ---
✅ 第 10 页完成。本页新增 10 个项目。耗时: 1.07 秒

--- 正在抓取第 11 页 ---
✅ 第 11 页完成。本页新增 10 个项目。耗时: 1.01 秒

--- 正在抓取第 12 页 ---
✅ 第 12 页完成。本页新增 10 个项目。耗时: 1.00 秒

--- 正在抓取第 13 页 ---
✅ 第 13 页完成。本页新增 10 个项目。耗时: 0.95 秒

--- 正在抓取第 14 页 ---
✅ 第 14 页完成。本页新增 10 个项目。耗时: 0.74 秒

--- 正在抓取第 15 页 ---
✅ 第 15 页完成。本页新增 10 个项目。耗时: 0.98 秒

--- 正在抓取第 16 页 ---
✅ 第 16 页完成。本页新增 9 个项目。耗时: 0.71 秒
已到达最后一页。
🎉 全部结束！
📊 总共抓取: 159 个项目
⏱️ 总耗时: 1 分 22 秒
⚡ 平均每页耗时: 5.15 秒
文件已保存为 hku_pro

,program_name,link
0,Faculty of Architecture Master of Architecture (Design) MArch (Design) Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(design)-foa&mode=0
1,Faculty of Architecture Master of Architecture (non-HK and International Applicants) MArch Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(non-hk-and-international-applicants)-foa&mode=0
2,Faculty of Architecture Master of Architecture (UGC funded for HK Applicants) MArch Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(ugc-funded-for-hk-applicants)-foa&mode=0
3,Faculty of Architecture Master of Digital Urban Management MDUM Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-digital-urban-management-foa&mode=0
4,Faculty of Architecture Master of Landscape Architecture MLA Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-landscape-architecture-foa&mode=0
5,Faculty of Architecture Master of Science in Advanced Architectural Design (January 2026) Closed MSc(AAD) Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-advanced-architectural-design-(sep-2026)-foa&mode=0
6,Faculty of Architecture Master of Science in Conservation (Conservation Planning and Management Stream) MSc(Conservation)CPM Full-Time & Part-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-conservation-foa&mode=0
7,Faculty of Architecture Master of Science in Conservation (Design for Conservation Stream) MSc(Con)Design Full-Time & Part-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-conservation-(design-for-conservation-stream)-foa&mode=0
8,Faculty of Architecture Master of Science in Construction Project Management MSc(ConstProjectMan) Full-Time & Part-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-construction-project-management-foa&mode=0
9,Faculty of Architecture Master of Science in Digital Management of Built Assets MSc(DMBA) Full-Time & Part-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-digital-management-of-built-assets-foa&mode=0


## Version 2

In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML

# --- 1. 爬虫设置与启动 ---
total_start_time = time.time()

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

target_url = "https://portal.hku.hk/tpg-admissions/programme-listing"
base_url = "https://portal.hku.hk"
all_valid_data = []

try:
    print(f"🔗 正在访问首页: {target_url}")
    driver.get(target_url)
    
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]'))
    )
    
    page_num = 1
    
    while True:
        print(f"\n📄 正在抓取第 {page_num} 页 ...")
        
        # 抓取逻辑
        current_links = driver.find_elements(By.TAG_NAME, "a")
        count_in_page = 0
        
        for link in current_links:
            try:
                url = link.get_attribute("href")
                text = link.text.strip()
                
                if url and "programme-details" in url and text:
                    full_url = urljoin(base_url, url)
                    if not any(d['项目官网链接'] == full_url for d in all_valid_data):
                        # 直接在这里使用最终的列名，方便后续处理
                        all_valid_data.append({
                            "项目名称": text,
                            "项目官网链接": full_url
                        })
                        count_in_page += 1
            except:
                continue
        
        print(f"   ✅ 本页新增 {count_in_page} 个项目")

        # 翻页逻辑
        try:
            next_buttons = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_buttons:
                print("   🛑 未找到下一页按钮，抓取结束。")
                break
            
            next_btn = next_buttons[0]
            parent_li = next_btn.find_element(By.XPATH, "./..")
            
            if "disabled" in parent_li.get_attribute("class"):
                print("   🛑 已到达最后一页。")
                break
                
            driver.execute_script("arguments[0].click();", next_btn)
            page_num += 1
            time.sleep(3) # 等待页面加载
            
        except Exception as e:
            print(f"   ⚠️ 翻页中断: {str(e)}")
            break

finally:
    driver.quit()
    print("\n🏁 浏览器已关闭。")

# --- 2. 数据处理与导出 (自动衔接，不会报错) ---

if all_valid_data:
    # 创建 DataFrame
    df = pd.DataFrame(all_valid_data)
    
    # 插入固定列 (学校信息)
    df.insert(0, "学校代码", "HK001")
    df.insert(1, "学校名称", "The University of Hong Kong")
    
    # 插入空白列
    df["项目申请链接"] = ""
    df["学生案例"] = ""
    df["面试问题"] = ""
    
    # 强制重新排序 (严格按照你的要求：从左到右)
    target_columns = [
        "学校代码", 
        "学校名称", 
        "项目名称", 
        "项目官网链接", 
        "项目申请链接", 
        "学生案例", 
        "面试问题"
    ]
    
    # 筛选并排序最终数据
    final_df = df[target_columns]
    
    # 导出文件
    file_name = "HKU_Final_List.csv"
    final_df.to_csv(file_name, index=False, encoding="utf-8-sig")
    
    # --- 统计与展示 ---
    total_time = time.time() - total_start_time
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)
    
    print("="*50)
    print(f"🎉 任务圆满完成！")
    print(f"📊 总共获取: {len(final_df)} 条数据")
    print(f"⏱️ 总耗时: {minutes}分 {seconds}秒")
    print(f"💾 文件已保存为: {file_name}")
    print("="*50)
    
    print("\n↓↓↓ 数据预览 (前5行) ↓↓↓")
    # 设置显示选项，确保链接完整显示
    pd.set_option('display.max_colwidth', None)
    display(final_df.head())

else:
    print("❌ 警告：未抓取到任何数据，无法生成文件。请检查网络或页面结构。")

🚀 正在启动浏览器...
🔗 正在访问首页: https://portal.hku.hk/tpg-admissions/programme-listing

📄 正在抓取第 1 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 2 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 3 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 4 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 5 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 6 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 7 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 8 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 9 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 10 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 11 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 12 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 13 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 14 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 15 页 ...
   ✅ 本页新增 10 个项目

📄 正在抓取第 16 页 ...
   ✅ 本页新增 9 个项目
   🛑 已到达最后一页。

🏁 浏览器已关闭。
🎉 任务圆满完成！
📊 总共获取: 159 条数据
⏱️ 总耗时: 1分 19秒
💾 文件已保存为: HKU_Final_List.csv

↓↓↓ 数据预览 (前5行) ↓↓↓


,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,学生案例,面试问题
0,HK001,The University of Hong Kong,Faculty of Architecture\nMaster of Architecture (Design)\nMArch (Design) Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(design)-foa&mode=0,,,
1,HK001,The University of Hong Kong,Faculty of Architecture\nMaster of Architecture (non-HK and International Applicants)\nMArch Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(non-hk-and-international-applicants)-foa&mode=0,,,
2,HK001,The University of Hong Kong,Faculty of Architecture\nMaster of Architecture (UGC funded for HK Applicants)\nMArch Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(ugc-funded-for-hk-applicants)-foa&mode=0,,,
3,HK001,The University of Hong Kong,Faculty of Architecture\nMaster of Digital Urban Management\nMDUM Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-digital-urban-management-foa&mode=0,,,
4,HK001,The University of Hong Kong,Faculty of Architecture\nMaster of Landscape Architecture\nMLA Full-Time,https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-landscape-architecture-foa&mode=0,,,


# 爬虫 12.11

# 📅 工作计划：HKU TPG 研究生项目深度爬取 (Deep Scraper)

**日期**：2025-12-11  
**目标**：从 HKU Taught Postgraduate Admissions 官网抓取完整项目列表，并深入详情页获取关键时间节点与真实申请链接。

---

## 1. 任务背景与需求变更
之前的爬虫仅抓取了项目名称和官网链接（Level 1），现在的需求升级为**二级深度抓取（Level 2）**，需要进入每一个项目的详情页获取以下核心信息：
* **项目开始时间 (Start Date)**
* **申请截止时间 (Deadline)**
* **真实的申请跳转链接 (Application URL)**：该链接通常隐藏在 "Apply Now" 按钮的点击动作之后，且会弹出新窗口。

## 2. 核心逻辑架构
本脚本采用 **“两阶段（Two-Phase）”** 策略以确保稳定性：

### 🟢 Phase 1：列表扫描 (List Scanning)
* **目标**：快速遍历 1-16 页的所有项目。
* **动作**：
    * 处理分页 (`Next` 按钮点击)。
    * 初步清洗项目名称（去除 Faculty 前缀）。
    * 去重并存储基础链接。

### 🔴 Phase 2：深度详情抓取 (Deep Detail Extraction)
* **目标**：逐个访问 Phase 1 获取的 URL，提取详细字段。
* **技术难点与解决方案**：
    1.  **动态时间提取**：使用 XPath `following-sibling` 定位，智能抓取 "Start Date" 和 "Deadline" 标签后的文本，适应不同页面的排版。
    2.  **申请链接获取 (Window Switching)**：
        * 模拟点击 `Apply Now` 按钮。
        * 检测是否弹出新窗口（Instruction Page）。
        * **切换句柄 (Switch Handle)** 至新窗口，抓取当前 URL。
        * 关闭新窗口，切回主列表，继续下一个循环。

---

## 3. 数据结构定义 (Output Schema)
最终导出数据将包含以下字段（Key），并按指定顺序排列：

| 字段名 | 说明 | 数据源 |
| :--- | :--- | :--- |
| **学校代码** | 固定值 | `HK001` |
| **学校名称** | 固定值 | `The University of Hong Kong` |
| **项目名称** | 清洗后的名称 | 列表页文本 (去除 Faculty 行) |
| **项目官网链接** | 详情页 URL | 列表页 `href` |
| **项目申请链接** | **关键目标** | 详情页点击 `Apply Now` 后跳转的 URL |
| **项目opendate** | 关键目标 | 详情页 `Start Date` 字段 |
| **项目deadline** | 关键目标 | 详情页 `Deadline` 字段 |
| **学生案例** | 预留空位 | (Empty) |
| **面试问题** | 预留空位 | (Empty) |

---

## 4. 运行注意事项
* **耗时预警**：由于涉及 150+ 次页面加载和模拟点击，预计运行时间较长（约 15-20 分钟）。
* **环境要求**：必须保持网络连接稳定，且不要手动关闭正在运行的浏览器窗口。

## Version 1

### crawler code

In [5]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display

# --- 1. 浏览器配置 ---
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--headless") # 需要看浏览器运行过程就保留注释，不需要看就取消注释
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"

# 全局变量：用于存储最终数据
full_details_data = []

try:
    # ==========================
    # 第一阶段：快速获取项目列表
    # ==========================
    print(f"Phase 1: 正在获取所有项目列表链接...")
    driver.get(list_url)
    
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    temp_links = []
    page_num = 1
    
    while True:
        print(f"  -> 正在扫描第 {page_num} 页...")
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    
                    # 清洗名称
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except:
                continue

        # 翻页逻辑
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"):
                break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(2)
            page_num += 1
        except:
            break
            
    print(f"✅ 列表获取完成！共找到 {len(temp_links)} 个项目。")
    print("="*50)

    # ==========================
    # 第二阶段：深度抓取详情
    # ==========================
    print(f"Phase 2: 开始深度抓取 (提取 opendate, deadline, 申请链接)...")

    for index, item in enumerate(temp_links):
        print(f"[{index+1}/{len(temp_links)}] 正在处理: {item['name'][:30]}...")
        
        start_date = "N/A"
        deadline = "N/A"
        apply_url = "N/A"
        
        try:
            driver.get(item['link'])
            time.sleep(1.5)
            
            # 提取 opendate (Start Date)
            try:
                start_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
                start_date = start_elem.text.strip()
            except:
                pass 

            # 提取 deadline
            try:
                dead_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
                deadline_text = dead_elem.text.strip()
                deadline = deadline_text.replace("\n", " | ")
            except:
                pass

            # 获取申请链接 (模拟点击 + 窗口切换)
            try:
                apply_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", apply_btn)
                time.sleep(2)
                
                all_windows = driver.window_handles
                if len(all_windows) > 1:
                    for handle in all_windows:
                        if handle != main_window:
                            driver.switch_to.window(handle)
                            apply_url = driver.current_url
                            driver.close()
                            break
                    driver.switch_to.window(main_window)
                else:
                    href = apply_btn.get_attribute("href")
                    if href: apply_url = href
            except:
                apply_url = ""

            # --- 关键修改：存入结果，使用新Key ---
            full_details_data.append({
                "学校代码": "HK001",
                "学校名称": "The University of Hong Kong",
                "项目名称": item['name'],
                "项目官网链接": item['link'],
                "项目申请链接": apply_url,
                "项目opendate": start_date,  # 已修改
                "项目deadline": deadline,    # 已修改
                "学生案例": "",
                "面试问题": ""
            })

        except Exception as e:
            print(f"❌ 失败: {item['name']} - {str(e)}")
            continue

finally:
    driver.quit()
    print("\n🏁 抓取任务结束！数据已存储在变量 'full_details_data' 中。")
    print(f"当前内存中共有 {len(full_details_data)} 条数据。请在下一个单元格进行导出。")

🚀 正在启动浏览器...
Phase 1: 正在获取所有项目列表链接...
  -> 正在扫描第 1 页...
  -> 正在扫描第 2 页...
  -> 正在扫描第 3 页...
  -> 正在扫描第 4 页...
  -> 正在扫描第 5 页...
  -> 正在扫描第 6 页...
  -> 正在扫描第 7 页...
  -> 正在扫描第 8 页...
  -> 正在扫描第 9 页...
  -> 正在扫描第 10 页...
  -> 正在扫描第 11 页...
  -> 正在扫描第 12 页...
  -> 正在扫描第 13 页...
  -> 正在扫描第 14 页...
  -> 正在扫描第 15 页...
  -> 正在扫描第 16 页...
✅ 列表获取完成！共找到 159 个项目。
Phase 2: 开始深度抓取 (提取 opendate, deadline, 申请链接)...
[1/159] 正在处理: Master of Architecture (Design...
[2/159] 正在处理: Master of Architecture (non-HK...
[3/159] 正在处理: Master of Architecture (UGC fu...
[4/159] 正在处理: Master of Digital Urban Manage...
[5/159] 正在处理: Master of Landscape Architectu...
[6/159] 正在处理: Master of Science in Advanced ...
[7/159] 正在处理: Master of Science in Conservat...
[8/159] 正在处理: Master of Science in Conservat...
[9/159] 正在处理: Master of Science in Construct...
[10/159] 正在处理: Master of Science in Digital M...
[11/159] 正在处理: Master of Science in Real Esta...
[12/159] 正在处理: Master of Science in Sustainab...
[13/159] 正在处理: Ma

### saving code

In [6]:
import pandas as pd
from IPython.display import display, HTML

# --- 1. 数据检查与转换 ---
if 'full_details_data' in locals() and full_details_data:
    print(f"✅ 检测到内存中有 {len(full_details_data)} 条数据，正在整理...")
    
    df = pd.DataFrame(full_details_data)
    
    # --- 2. 格式标准化 (确保列齐全) ---
    # 定义标准列顺序
    target_columns = [
        "学校代码", 
        "学校名称", 
        "项目名称", 
        "项目官网链接", 
        "项目申请链接", 
        "项目opendate", 
        "项目deadline", 
        "学生案例", 
        "面试问题"
    ]
    
    # 自动填充缺失的列 (比如你是从内存直接读的，可能缺少固定的学校信息)
    if "学校代码" not in df.columns:
        df["学校代码"] = "HK001"
    if "学校名称" not in df.columns:
        df["学校名称"] = "The University of Hong Kong"
    
    # 填充空白列
    for col in ["学生案例", "面试问题"]:
        if col not in df.columns:
            df[col] = ""
            
    # 智能处理：确保所有需要的列都在，并且按顺序排列
    # 使用 reindex 会自动把缺少的列填 NaN，我们用 fillna('') 变为空白
    df_final = df.reindex(columns=target_columns).fillna("")
    
    # --- 3. 导出文件 ---
    csv_filename = "HKU_Full_Project_List.csv"
    excel_filename = "HKU_Full_Project_List.xlsx"
    
    # 导出 CSV (通用)
    df_final.to_csv(csv_filename, index=False, encoding="utf-8-sig")
    
    # 导出 Excel (如果安装了 openpyxl，推荐用这个，格式更好)
    try:
        df_final.to_excel(excel_filename, index=False)
        print(f"💾 Excel 文件已保存: {excel_filename}")
    except:
        print("⚠️ 未检测到 openpyxl 库，仅保存了 CSV 文件。")
        
    print(f"💾 CSV 文件已保存: {csv_filename}")
    
    # --- 4. 交互式预览 (点击链接跳转) ---
    print("\n↓↓↓ 数据预览 (点击链接可直接跳转) ↓↓↓")
    
    # 设置 pandas 显示选项
    pd.set_option('display.max_colwidth', None) # 让链接显示全
    
    def make_clickable(val):
        # 如果是链接格式，转为 HTML 超链接
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
    
    # 渲染 HTML 表格
    # 对 "项目官网链接" 和 "项目申请链接" 应用点击跳转
    styled_df = df_final.style.format({
        '项目官网链接': make_clickable,
        '项目申请链接': make_clickable
    })
    
    display(HTML(styled_df.to_html(escape=False)))

else:
    print("❌ 错误：内存中未找到 'full_details_data' 变量。")
    print("请确保上一步的爬虫代码已经运行完毕，并且不要重启 Jupyter Kernel。")

✅ 检测到内存中有 159 条数据，正在整理...
💾 Excel 文件已保存: HKU_Full_Project_List.xlsx
💾 CSV 文件已保存: HKU_Full_Project_List.csv

↓↓↓ 数据预览 (点击链接可直接跳转) ↓↓↓


,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Architecture (Design),点击跳转,点击跳转,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Architecture (UGC funded for HK Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
3,HK001,The University of Hong Kong,Master of Digital Urban Management,点击跳转,点击跳转,September 2026,"Early Round Admission: | 12:00 noon (GMT +8), October 22, 2025 | Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
4,HK001,The University of Hong Kong,Master of Landscape Architecture,点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
5,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026) Closed,点击跳转,,January 2026,"Main Round: | 12:00 noon (GMT +8), November 03, 2025",,
6,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
7,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
8,HK001,The University of Hong Kong,Master of Science in Construction Project Management,点击跳转,点击跳转,September 2026,"Early Admissions Round will take place from September to December 2025, with applications reviewed by the Department on a rolling basis. Those who are not admitted in the early round will be considered in the Main Round.: | 12:00 noon (GMT +8), January 02, 2026",,
9,HK001,The University of Hong Kong,Master of Science in Digital Management of Built Assets,点击跳转,点击跳转,September 2026,"Early Admissions Round will take place from September to December 2025, with applications reviewed by the Department on a rolling basis. Those who are not admitted in the early round will be considered in the Main Round with the application deadline: | 12:00 noon (GMT +8), January 02, 2026",,


## Version 2

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML, clear_output

# --- 1. 浏览器高级配置 ---
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--headless") # 调试阶段建议保留界面，观察跳转逻辑是否正确
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"

# 全局变量存储结果
full_details_data = []
total_start_time = time.time()

try:
    # ==========================================
    # Phase 1: 极速扫描列表 (不变，快速获取入口)
    # ==========================================
    print(f"Phase 1: 正在初始化项目列表...")
    driver.get(list_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    temp_links = []
    page_num = 1
    
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except:
                continue

        # 翻页
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"): break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
            page_num += 1
        except:
            break
            
    print(f"✅ Phase 1 完成！共锁定 {len(temp_links)} 个目标项目。")
    time.sleep(1)

    # ==========================================
    # Phase 2: 深度详情抓取 (包含“三级跳”逻辑)
    # ==========================================
    print(f"Phase 2: 开始深度抓取 (含 Applying 二次跳转)...")
    
    for index, item in enumerate(temp_links):
        item_start_time = time.time() # 单条计时开始
        
        # 初始化数据
        current_data = {
            "学校代码": "HK001",
            "学校名称": "The University of Hong Kong",
            "项目名称": item['name'],
            "项目官网链接": item['link'],
            "项目opendate": "N/A",
            "项目deadline": "N/A",
            "项目申请链接": "N/A", # 最终目标
            "学生案例": "",
            "面试问题": ""
        }
        
        try:
            # 1. 进入详情页
            driver.get(item['link'])
            # 稍作等待确保内容加载
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Expected Duration')]")))
            
            # 2. 抓取时间信息
            try:
                start_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
                current_data["项目opendate"] = start_elem.text.strip()
            except: pass

            try:
                dead_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
                current_data["项目deadline"] = dead_elem.text.strip().replace("\n", " | ")
            except: pass

            # 3. 抓取申请链接 (复杂的三级跳逻辑)
            try:
                # 步骤 A: 点击 "Apply Now"
                apply_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", apply_btn)
                time.sleep(2) # 等待新窗口弹出
                
                # 检查窗口变化
                all_windows = driver.window_handles
                if len(all_windows) > 1:
                    # 步骤 B: 切换到 "Instruction Page" (说明页)
                    for handle in all_windows:
                        if handle != main_window:
                            driver.switch_to.window(handle)
                            break
                    
                    # 步骤 C: 在说明页点击 "Applying"
                    # 这里我们要找类似 bullet point 的链接
                    try:
                        # 等待 Applying 链接出现
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'Applying')]"))
                        )
                        applying_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Applying')]")
                        
                        # 点击 Applying
                        driver.execute_script("arguments[0].click();", applying_link)
                        time.sleep(3) # 等待跳转或新窗口
                        
                        # 再次检查窗口：有时候 Applying 会在当前页跳转，有时候会再弹窗
                        current_windows_after_click = driver.window_handles
                        if len(current_windows_after_click) > len(all_windows):
                             # 如果又弹了一个窗，那才是真的申请页
                            newest_window = current_windows_after_click[-1]
                            driver.switch_to.window(newest_window)
                            final_url = driver.current_url
                            driver.close() # 关闭申请页
                            driver.switch_to.window(all_windows[-1]) # 回到说明页
                        else:
                            # 如果是在当前页跳转
                            final_url = driver.current_url
                        
                        current_data["项目申请链接"] = final_url
                        
                    except Exception as inner_e:
                        # 如果找不到 Applying，或者点击报错，说明可能直接就是申请页，或者结构不同
                        # 兜底：记录当前说明页的 URL
                        print(f"   ⚠️ 未找到 'Applying' 跳转，保留当前页链接: {inner_e}")
                        current_data["项目申请链接"] = driver.current_url
                    
                    # 关闭说明页，切回主窗口
                    driver.close()
                    driver.switch_to.window(main_window)
                else:
                    # 如果没有弹窗，直接获取 href
                    current_data["项目申请链接"] = apply_btn.get_attribute("href")

            except Exception as e:
                print(f"   ⚠️ 链接获取步骤异常: {str(e)}")

        except Exception as e:
            print(f"❌ 处理失败: {item['name']} - {str(e)}")
        
        # 存入结果
        full_details_data.append(current_data)
        
        # 计算单条耗时
        item_end_time = time.time()
        duration = item_end_time - item_start_time
        
        # 实时打印进度
        print(f"[{index+1}/{len(temp_links)}] 完成: {item['name'][:20]}... | ⏱️ 耗时: {duration:.2f}s")

finally:
    driver.quit()
    total_end_time = time.time()
    total_duration = total_end_time - total_start_time
    minutes = int(total_duration // 60)
    seconds = int(total_duration % 60)
    
    print("\n" + "="*50)
    print(f"🎉 抓取任务全部结束！")
    print(f"⏱️ 总耗时: {minutes} 分 {seconds} 秒")
    print(f"📊 获取数据: {len(full_details_data)} 条")
    print("="*50)

# --- 4. 结果可视化预览 (不生成文件，直接看) ---
if full_details_data:
    df_result = pd.DataFrame(full_details_data)
    
    # 确保列顺序正确
    cols = ["学校代码", "学校名称", "项目名称", "项目官网链接", "项目申请链接", "项目opendate", "项目deadline", "学生案例", "面试问题"]
    df_result = df_result[cols]
    
    print("\n↓↓↓ 爬取内容预览 (Top 10) ↓↓↓")
    
    # 设置样式：左对齐，链接可点击
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
        
    styled_table = df_result.head(10).style.format({
        '项目官网链接': make_clickable,
        '项目申请链接': make_clickable
    }).set_properties(**{'text-align': 'left'})
    
    display(HTML(styled_table.to_html(escape=False)))
else:
    print("❌ 没有数据可展示")

🚀 正在启动浏览器...
Phase 1: 正在初始化项目列表...
✅ Phase 1 完成！共锁定 159 个目标项目。
Phase 2: 开始深度抓取 (含 Applying 二次跳转)...
   ⚠️ 未找到 'Applying' 跳转，保留当前页链接: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x560d13
	0x560d54
	0x34e6dd
	0x3993a5
	0x39977b
	0x3e0382
	0x3bb534
	0x3ddb13
	0x3bb2e6
	0x38d321
	0x38e1d4
	0x7b4d54
	0x7b030b
	0x7ccbea
	0x57ac18
	0x582c1d
	0x569018
	0x5691d9
	0x553568
	0x76d85d49
	0x774dd6db
	0x774dd661

[1/159] 完成: Master of Architectu... | ⏱️ 耗时: 15.54s
   ⚠️ 未找到 'Applying' 跳转，保留当前页链接: Message: 
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x560d13
	0x560d54
	0x34e6dd
	0x3993a5
	0x39977b
	0x3e0382
	0x3bb534
	0x3ddb13
	0x3bb2e6
	0x38d321
	0x38e1d4
	0x7b4d54
	0x7b030b
	0x7ccbea
	0x57ac18
	0x582c1d
	0x569018
	0x5691d9
	0x553568
	0x76d85d49
	0x774dd6db
	0x774dd661

[2/159] 完成: Master of Architectu... | ⏱️ 耗时: 15.36s
❌ 处理失败: Master of Architecture (UGC funded for HK Applicants) - Message: no such window: target window already closed
from unk

,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Architecture (Design),点击跳转,点击跳转,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Architecture (UGC funded for HK Applicants),点击跳转,N/A,N/A,N/A,,
3,HK001,The University of Hong Kong,Master of Digital Urban Management,点击跳转,N/A,N/A,N/A,,
4,HK001,The University of Hong Kong,Master of Landscape Architecture,点击跳转,N/A,N/A,N/A,,
5,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026) Closed,点击跳转,N/A,N/A,N/A,,
6,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),点击跳转,N/A,N/A,N/A,,
7,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),点击跳转,N/A,N/A,N/A,,
8,HK001,The University of Hong Kong,Master of Science in Construction Project Management,点击跳转,N/A,N/A,N/A,,
9,HK001,The University of Hong Kong,Master of Science in Digital Management of Built Assets,点击跳转,N/A,N/A,N/A,,


## Version 3

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML

# --- 1. 浏览器高级配置 ---
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--headless") # 建议保留界面，观察跳转
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"

full_details_data = []
total_start_time = time.time()

try:
    # ================= Phase 1: 极速获取列表 (保持不变) =================
    print(f"Phase 1: 正在初始化项目列表...")
    driver.get(list_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    temp_links = []
    page_num = 1
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except: continue
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"): break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
            page_num += 1
        except: break
    print(f"✅ Phase 1 完成！共锁定 {len(temp_links)} 个目标项目。")
    time.sleep(1)

    # ================= Phase 2: 深度详情抓取 (核心修复) =================
    print(f"Phase 2: 开始深度抓取 (已增强 Applying 识别能力)...")
    
    for index, item in enumerate(temp_links):
        item_start_time = time.time()
        
        current_data = {
            "学校代码": "HK001",
            "学校名称": "The University of Hong Kong",
            "项目名称": item['name'],
            "项目官网链接": item['link'],
            "项目opendate": "N/A",
            "项目deadline": "N/A",
            "项目申请链接": "N/A",
            "学生案例": "",
            "面试问题": ""
        }
        
        try:
            driver.get(item['link'])
            # 缩短等待，提高效率，只要有 Start Date 就可以开始了
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Start Date')]")))
            
            # --- 抓取时间 ---
            try:
                start_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
                current_data["项目opendate"] = start_elem.text.strip()
            except: pass
            
            try:
                dead_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
                current_data["项目deadline"] = dead_elem.text.strip().replace("\n", " | ")
            except: pass

            # --- 抓取申请链接 (修复版) ---
            try:
                # 1. 点击 Apply Now
                apply_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", apply_btn)
                time.sleep(2) # 等待弹窗
                
                all_windows = driver.window_handles
                if len(all_windows) > 1:
                    # 2. 切换到说明页
                    driver.switch_to.window(all_windows[-1])
                    
                    # 3. ★★★ 核心修复：查找 Applying 链接 ★★★
                    # 策略：同时找文字包含'Applying' 或者 链接包含'CreateUserScreen' (最稳)
                    try:
                        # 显式等待：直到元素可被点击
                        applying_link = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Applying')] | //a[contains(@href, 'CreateUserScreen')]"))
                        )
                        
                        # 获取链接目标，如果不为空，直接用
                        target_href = applying_link.get_attribute("href")
                        
                        # 点击它 (有时候点击会变URL，有时候会弹新窗，为了保险我们先拿href)
                        if target_href and "http" in target_href:
                            current_data["项目申请链接"] = target_href
                        else:
                            # 如果 href 是空的 js 调用，则必须点击
                            driver.execute_script("arguments[0].click();", applying_link)
                            time.sleep(3)
                            # 如果弹了第三个窗
                            if len(driver.window_handles) > len(all_windows):
                                driver.switch_to.window(driver.window_handles[-1])
                                current_data["项目申请链接"] = driver.current_url
                                driver.close()
                                driver.switch_to.window(all_windows[-1])
                            else:
                                current_data["项目申请链接"] = driver.current_url
                                
                    except Exception as e:
                        # 兜底：如果实在找不到 Applying，就记录当前说明页链接，总比没有好
                        print(f"   ⚠️ 找不到 Applying 按钮，记录当前页: {str(e)[:50]}")
                        current_data["项目申请链接"] = driver.current_url
                    
                    # 关闭说明页，切回主页
                    driver.close()
                    driver.switch_to.window(main_window)
                else:
                    # 无弹窗，直接拿 href
                    current_data["项目申请链接"] = apply_btn.get_attribute("href")

            except Exception as e:
                print(f"   ⚠️ 链接步骤异常: {str(e)[:50]}")

        except Exception as e:
            print(f"❌ 项目异常: {item['name'][:10]} - {str(e)[:50]}")
        
        full_details_data.append(current_data)
        
        # 计时与显示
        duration = time.time() - item_start_time
        print(f"[{index+1}/{len(temp_links)}] 完成: {item['name'][:25]}... | ⏱️ {duration:.1f}s")

finally:
    driver.quit()
    total_time = time.time() - total_start_time
    print(f"\n🎉 抓取结束！总耗时: {total_time/60:.1f} 分钟。共 {len(full_details_data)} 条数据。")

# --- 可视化预览 ---
if full_details_data:
    df_result = pd.DataFrame(full_details_data)
    cols = ["学校代码", "学校名称", "项目名称", "项目官网链接", "项目申请链接", "项目opendate", "项目deadline", "学生案例", "面试问题"]
    df_result = df_result[cols]
    
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
        
    print("\n↓↓↓ 数据预览 (点击链接验证) ↓↓↓")
    # 简化的样式调用，兼容性最强
    display(HTML(df_result.head(10).style.format({'项目官网链接': make_clickable, '项目申请链接': make_clickable}).to_html(escape=False)))

🚀 正在启动浏览器...
Phase 1: 正在初始化项目列表...
✅ Phase 1 完成！共锁定 159 个目标项目。
Phase 2: 开始深度抓取 (已增强 Applying 识别能力)...
   ⚠️ 找不到 Applying 按钮，记录当前页: Message: 
Stacktrace:
Symbols not available. Dumpi
[1/159] 完成: Master of Architecture (D... | ⏱️ 16.6s
   ⚠️ 找不到 Applying 按钮，记录当前页: Message: 
Stacktrace:
Symbols not available. Dumpi
[2/159] 完成: Master of Architecture (n... | ⏱️ 23.5s
   ⚠️ 找不到 Applying 按钮，记录当前页: Message: 
Stacktrace:
Symbols not available. Dumpi
[3/159] 完成: Master of Architecture (U... | ⏱️ 28.8s
   ⚠️ 找不到 Applying 按钮，记录当前页: Message: 
Stacktrace:
Symbols not available. Dumpi
[4/159] 完成: Master of Digital Urban M... | ⏱️ 24.2s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[5/159] 完成: Master of Landscape Archi... | ⏱️ 5.5s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[6/159] 完成: Master of Science in Adva... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[7/159] 完成: Master of Science in Cons... | ⏱️ 0.0s
❌

,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Architecture (Design),点击跳转,点击跳转,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Architecture (UGC funded for HK Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
3,HK001,The University of Hong Kong,Master of Digital Urban Management,点击跳转,点击跳转,September 2026,"Early Round Admission: | 12:00 noon (GMT +8), October 22, 2025 | Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
4,HK001,The University of Hong Kong,Master of Landscape Architecture,点击跳转,N/A,N/A,N/A,,
5,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026) Closed,点击跳转,N/A,N/A,N/A,,
6,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),点击跳转,N/A,N/A,N/A,,
7,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),点击跳转,N/A,N/A,N/A,,
8,HK001,The University of Hong Kong,Master of Science in Construction Project Management,点击跳转,N/A,N/A,N/A,,
9,HK001,The University of Hong Kong,Master of Science in Digital Management of Built Assets,点击跳转,N/A,N/A,N/A,,


## Version 4

In [3]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML

# --- 1. 浏览器高级配置 ---
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
# chrome_options.add_argument("--headless") # 调试时建议保持开启界面，看它点击
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

print("🚀 正在启动浏览器...")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"

full_details_data = []
total_start_time = time.time()

try:
    # ================= Phase 1: 极速获取列表 =================
    print(f"Phase 1: 正在初始化项目列表...")
    driver.get(list_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    temp_links = []
    page_num = 1
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except: continue
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"): break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
            page_num += 1
        except: break
    print(f"✅ Phase 1 完成！共锁定 {len(temp_links)} 个目标项目。")
    time.sleep(1)

    # ================= Phase 2: 深度详情抓取 (针对 onclick 修复) =================
    print(f"Phase 2: 开始深度抓取 (精准定位 ID='a_application')...")
    
    for index, item in enumerate(temp_links):
        item_start_time = time.time()
        
        current_data = {
            "学校代码": "HK001",
            "学校名称": "The University of Hong Kong",
            "项目名称": item['name'],
            "项目官网链接": item['link'],
            "项目opendate": "N/A",
            "项目deadline": "N/A",
            "项目申请链接": "N/A",
            "学生案例": "",
            "面试问题": ""
        }
        
        try:
            driver.get(item['link'])
            # 缩短等待
            WebDriverWait(driver, 8).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Start Date')]")))
            
            # --- 抓取时间 (保持不变) ---
            try:
                start_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
                current_data["项目opendate"] = start_elem.text.strip()
            except: pass
            
            try:
                dead_elem = driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
                current_data["项目deadline"] = dead_elem.text.strip().replace("\n", " | ")
            except: pass

            # --- 抓取申请链接 (精准 ID 版) ---
            try:
                # 1. 点击 Apply Now
                apply_btn = driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
                main_window = driver.current_window_handle
                driver.execute_script("arguments[0].click();", apply_btn)
                time.sleep(2) # 等待 说明页 弹出
                
                windows_step1 = driver.window_handles
                if len(windows_step1) > 1:
                    # 2. 切换到说明页
                    driver.switch_to.window(windows_step1[-1])
                    
                    # 3. ★★★ 核心修复：直接找 ID="a_application" ★★★
                    # 不再找文字，直接找这个独特的 ID 里的 a 标签
                    try:
                        # 等待该元素出现
                        applying_btn = WebDriverWait(driver, 5).until(
                            EC.element_to_be_clickable((By.CSS_SELECTOR, "#a_application a"))
                        )
                        
                        # 因为它是 onclick 触发 JS，必须点击它
                        # 使用 JS 点击最稳，能绕过页面上可能存在的遮挡
                        driver.execute_script("arguments[0].click();", applying_btn)
                        time.sleep(3) # 等待 真正的申请页 弹出
                        
                        # 4. 检查是否弹出了第三个窗口
                        windows_step2 = driver.window_handles
                        if len(windows_step2) > len(windows_step1):
                            # 弹出了新窗，这就是最终目标
                            final_window = windows_step2[-1]
                            driver.switch_to.window(final_window)
                            current_data["项目申请链接"] = driver.current_url
                            
                            # 关掉最终页
                            driver.close()
                            # 切回说明页
                            driver.switch_to.window(windows_step1[-1])
                        else:
                            # 没弹窗？可能是当前页刷新了，直接拿当前页URL
                            current_data["项目申请链接"] = driver.current_url
                            
                    except Exception as e:
                        print(f"   ⚠️ ID定位失败: {str(e)[:50]}")
                        # 兜底：记录当前说明页
                        current_data["项目申请链接"] = driver.current_url
                    
                    # 关掉说明页
                    driver.close()
                    # 切回主页面
                    driver.switch_to.window(main_window)
                else:
                    # 没有说明页，直接就是申请页
                    current_data["项目申请链接"] = apply_btn.get_attribute("href")

            except Exception as e:
                print(f"   ⚠️ 跳转流程异常: {str(e)[:50]}")

        except Exception as e:
            print(f"❌ 项目异常: {item['name'][:10]} - {str(e)[:50]}")
        
        full_details_data.append(current_data)
        
        duration = time.time() - item_start_time
        print(f"[{index+1}/{len(temp_links)}] 完成: {item['name'][:25]}... | ⏱️ {duration:.1f}s")

finally:
    driver.quit()
    total_time = time.time() - total_start_time
    print(f"\n🎉 抓取结束！总耗时: {total_time/60:.1f} 分钟。共 {len(full_details_data)} 条数据。")

# --- 结果可视化 ---
if full_details_data:
    df_result = pd.DataFrame(full_details_data)
    cols = ["学校代码", "学校名称", "项目名称", "项目官网链接", "项目申请链接", "项目opendate", "项目deadline", "学生案例", "面试问题"]
    df_result = df_result[cols]
    
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
        
    print("\n↓↓↓ 数据预览 (点击链接验证) ↓↓↓")
    display(HTML(df_result.head(10).style.format({'项目官网链接': make_clickable, '项目申请链接': make_clickable}).to_html(escape=False)))

🚀 正在启动浏览器...
Phase 1: 正在初始化项目列表...
✅ Phase 1 完成！共锁定 159 个目标项目。
Phase 2: 开始深度抓取 (精准定位 ID='a_application')...
[1/159] 完成: Master of Architecture (D... | ⏱️ 27.3s
[2/159] 完成: Master of Architecture (n... | ⏱️ 16.3s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[3/159] 完成: Master of Architecture (U... | ⏱️ 3.6s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[4/159] 完成: Master of Digital Urban M... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[5/159] 完成: Master of Landscape Archi... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[6/159] 完成: Master of Science in Adva... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[7/159] 完成: Master of Science in Cons... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: target window already clo
[8/159] 完成: Master of Science in Cons... | ⏱️ 0.0s
❌ 项目异常: Master of  - Message: no such window: targ

,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Architecture (Design),点击跳转,点击跳转,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Architecture (UGC funded for HK Applicants),点击跳转,N/A,N/A,N/A,,
3,HK001,The University of Hong Kong,Master of Digital Urban Management,点击跳转,N/A,N/A,N/A,,
4,HK001,The University of Hong Kong,Master of Landscape Architecture,点击跳转,N/A,N/A,N/A,,
5,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026) Closed,点击跳转,N/A,N/A,N/A,,
6,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),点击跳转,N/A,N/A,N/A,,
7,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),点击跳转,N/A,N/A,N/A,,
8,HK001,The University of Hong Kong,Master of Science in Construction Project Management,点击跳转,N/A,N/A,N/A,,
9,HK001,The University of Hong Kong,Master of Science in Digital Management of Built Assets,点击跳转,N/A,N/A,N/A,,


## Version 5

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import urljoin
from IPython.display import display, HTML
from concurrent.futures import ThreadPoolExecutor, as_completed

# ==============================================================================
# 🟢【用户配置区】在这里修改并发数量
# ==============================================================================
MAX_WORKERS = 8   # 建议设置 3-8。设置越大越快，但对电脑负载越高。默认 5。
# ==============================================================================

# 全局变量
base_url = "https://portal.hku.hk"
list_url = "https://portal.hku.hk/tpg-admissions/programme-listing"
total_start_time = time.time()

# --- 1. 浏览器驱动生成器 (默认开启无头模式) ---
def get_driver(headless=True):
    chrome_options = Options()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    # ⬇️ 这一行控制是否后台运行。True=后台(看不见)，False=前台(看得见)
    if headless:
        chrome_options.add_argument("--headless") 
        
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    # 优化：禁止加载图片，提升后台抓取速度
    prefs = {"profile.managed_default_content_settings.images": 2}
    chrome_options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# --- 2. Phase 1: 快速获取列表 (单线程) ---
print(f"🚀 [Phase 1] 正在扫描任务列表...")
driver = get_driver(headless=True) # 列表抓取也静默运行
temp_links = []

try:
    driver.get(list_url)
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[href*="programme-details"]')))
    
    while True:
        elements = driver.find_elements(By.CSS_SELECTOR, 'a[href*="programme-details"]')
        for elem in elements:
            try:
                link = elem.get_attribute("href")
                raw_text = elem.text.strip()
                if link and raw_text:
                    full_link = urljoin(base_url, link)
                    lines = raw_text.split('\n')
                    prog_name = lines[1].strip() if len(lines) > 1 else lines[0].strip()
                    if not any(d['link'] == full_link for d in temp_links):
                        temp_links.append({"name": prog_name, "link": full_link})
            except: continue
        try:
            next_btns = driver.find_elements(By.XPATH, "//a[contains(text(), '»')]")
            if not next_btns: break
            btn = next_btns[0]
            if "disabled" in btn.find_element(By.XPATH, "./..").get_attribute("class"): break
            driver.execute_script("arguments[0].click();", btn)
            time.sleep(1)
        except: break
finally:
    driver.quit()

print(f"✅ [Phase 1] 完成！共锁定 {len(temp_links)} 个任务。")
print(f"🔥 [Phase 2] 启动 {MAX_WORKERS} 个并发窗口进行后台抓取...")
time.sleep(1)

# --- 3. Phase 2: 分布式抓取逻辑 (单个Worker) ---
def process_single_item(item):
    item_start = time.time() # 开始计时
    local_driver = get_driver(headless=True) # 强制后台运行
    
    result = {
        "学校代码": "HK001",
        "学校名称": "The University of Hong Kong",
        "项目名称": item['name'],
        "项目官网链接": item['link'],
        "项目opendate": "N/A",
        "项目deadline": "N/A",
        "项目申请链接": "N/A",
        "学生案例": "",
        "面试问题": ""
    }
    
    try:
        local_driver.get(item['link'])
        WebDriverWait(local_driver, 15).until(EC.presence_of_element_located((By.XPATH, "//*[contains(text(), 'Start Date')]")))
        
        # 抓取时间
        try:
            start_elem = local_driver.find_element(By.XPATH, "//*[contains(text(), 'Start Date')]/following-sibling::*")
            result["项目opendate"] = start_elem.text.strip()
        except: pass
        
        try:
            dead_elem = local_driver.find_element(By.XPATH, "//*[contains(text(), 'Deadline')]/following-sibling::*")
            result["项目deadline"] = dead_elem.text.strip().replace("\n", " | ")
        except: pass

        # 抓取申请链接 (精准ID定位法)
        try:
            apply_btn = local_driver.find_element(By.XPATH, "//a[contains(text(), 'Apply Now')]")
            main_window = local_driver.current_window_handle
            local_driver.execute_script("arguments[0].click();", apply_btn)
            time.sleep(2)
            
            windows_step1 = local_driver.window_handles
            if len(windows_step1) > 1:
                local_driver.switch_to.window(windows_step1[-1])
                try:
                    # 直接找 ID="a_application"
                    applying_btn = WebDriverWait(local_driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "#a_application a"))
                    )
                    local_driver.execute_script("arguments[0].click();", applying_btn)
                    time.sleep(3)
                    
                    windows_step2 = local_driver.window_handles
                    if len(windows_step2) > len(windows_step1):
                        local_driver.switch_to.window(windows_step2[-1])
                        result["项目申请链接"] = local_driver.current_url
                        local_driver.close()
                        local_driver.switch_to.window(windows_step1[-1])
                    else:
                        result["项目申请链接"] = local_driver.current_url
                except:
                    # 兜底：如果没找到ID，就记录当前说明页链接
                    result["项目申请链接"] = local_driver.current_url
                
                local_driver.close()
                local_driver.switch_to.window(main_window)
            else:
                result["项目申请链接"] = apply_btn.get_attribute("href")
        except: pass
        
    except Exception as e:
        print(f"❌ 抓取失败: {item['name'][:10]} - {e}")
    finally:
        local_driver.quit() # 释放内存
        
    # 计算单条耗时
    duration = time.time() - item_start
    return result, duration

# --- 4. 线程池执行与监控 ---
full_details_data = []
completed_count = 0

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # 提交任务
    future_to_item = {executor.submit(process_single_item, item): item for item in temp_links}
    
    print(f"⏳ 任务队列已建立，正在全力运行中 (请留意下方进度)...")
    
    for future in as_completed(future_to_item):
        item = future_to_item[future]
        try:
            data, duration = future.result()
            full_details_data.append(data)
            completed_count += 1
            
            # 实时打印单条耗时
            print(f"[{completed_count}/{len(temp_links)}] ✅ 完成: {data['项目名称'][:20]}... | ⏱️ 耗时: {duration:.2f}s")
            
        except Exception as exc:
            print(f"❌ 任务异常: {item['name']} generated an exception: {exc}")

# --- 5. 结果可视化展示 ---
total_time = time.time() - total_start_time
print(f"\n" + "="*50)
print(f"🎉 全部结束！总耗时: {total_time/60:.2f} 分钟")
print(f"📊 成功获取: {len(full_details_data)} 条数据")
print("="*50)

if full_details_data:
    df_result = pd.DataFrame(full_details_data)
    
    cols = ["学校代码", "学校名称", "项目名称", "项目官网链接", "项目申请链接", "项目opendate", "项目deadline", "学生案例", "面试问题"]
    for c in cols:
        if c not in df_result.columns: df_result[c] = ""
    df_result = df_result[cols]
    
    pd.set_option('display.max_colwidth', None)
    
    def make_clickable(val):
        if val and str(val).startswith("http"):
            return f'<a target="_blank" href="{val}">点击跳转</a>'
        return val
        
    print("\n↓↓↓ 最终抓取结果预览 (点击链接验证) ↓↓↓")
    display(HTML(df_result.head(10).style.format({'项目官网链接': make_clickable, '项目申请链接': make_clickable}).to_html(escape=False)))

🚀 [Phase 1] 正在扫描任务列表...
✅ [Phase 1] 完成！共锁定 159 个任务。
🔥 [Phase 2] 启动 8 个并发窗口进行后台抓取...
⏳ 任务队列已建立，正在全力运行中 (请留意下方进度)...
[1/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 18.60s
[2/159] ✅ 完成: Master of Architectu... | ⏱️ 耗时: 28.89s
[3/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 29.35s
[4/159] ✅ 完成: Master of Architectu... | ⏱️ 耗时: 30.78s
[5/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 31.07s
[6/159] ✅ 完成: Master of Architectu... | ⏱️ 耗时: 31.58s
[7/159] ✅ 完成: Master of Digital Ur... | ⏱️ 耗时: 34.52s
[8/159] ✅ 完成: Master of Landscape ... | ⏱️ 耗时: 44.04s
[9/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 28.68s
[10/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 29.15s
[11/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 31.51s
[12/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 34.16s
[13/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 30.37s
[14/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 44.45s
[15/159] ✅ 完成: Master of Science in... | ⏱️ 耗时: 50.57s
[16/159] ✅ 完成: Master of Urban Desi... | ⏱️ 耗时: 33.29s
[17/159] ✅ 完成:

,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026),点击跳转,N/A,January 2026,"Main Round: | 12:00 noon (GMT +8), November 03, 2025",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
3,HK001,The University of Hong Kong,Master of Architecture (Design),点击跳转,点击跳转,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
4,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
5,HK001,The University of Hong Kong,Master of Architecture (UGC funded for HK Applicants),点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
6,HK001,The University of Hong Kong,Master of Digital Urban Management,点击跳转,点击跳转,September 2026,"Early Round Admission: | 12:00 noon (GMT +8), October 22, 2025 | Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
7,HK001,The University of Hong Kong,Master of Landscape Architecture,点击跳转,点击跳转,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
8,HK001,The University of Hong Kong,Master of Science in Construction Project Management,点击跳转,点击跳转,September 2026,"Early Admissions Round will take place from September to December 2025, with applications reviewed by the Department on a rolling basis. Those who are not admitted in the early round will be considered in the Main Round.: | 12:00 noon (GMT +8), January 02, 2026",,
9,HK001,The University of Hong Kong,Master of Science in Digital Management of Built Assets,点击跳转,点击跳转,September 2026,"Early Admissions Round will take place from September to December 2025, with applications reviewed by the Department on a rolling basis. Those who are not admitted in the early round will be considered in the Main Round with the application deadline: | 12:00 noon (GMT +8), January 02, 2026",,


In [2]:
import pandas as pd

# 1. 检查是否有数据
if 'full_details_data' in locals() and full_details_data:
    print(f"🔄 正在整理 {len(full_details_data)} 条数据...")
    
    # 2. 创建 DataFrame
    df = pd.DataFrame(full_details_data)
    
    # 3. 强制规定列顺序 (严格按照你的要求)
    target_columns = [
        "学校代码", 
        "学校名称", 
        "项目名称", 
        "项目官网链接", 
        "项目申请链接", 
        "项目opendate", 
        "项目deadline", 
        "学生案例", 
        "面试问题"
    ]
    
    # 4. 自动补全缺失的列 (比如'学生案例'这种空的)
    for col in target_columns:
        if col not in df.columns:
            df[col] = ""
            
    # 5. 重新排序和筛选
    df_final = df[target_columns]
    
    # 6. 定义文件名
    excel_filename = "HKU_Projects_Final.xlsx"
    
    # 7. 导出 Excel
    try:
        # index=False 表示不保存左边那一列 0,1,2,3 的索引号
        df_final.to_excel(excel_filename, index=False)
        print("=" * 50)
        print(f"✅ 成功导出 Excel 文件！")
        print(f"📂 文件名: {excel_filename}")
        print(f"📊 包含数据: {len(df_final)} 行")
        print("=" * 50)
        print("💡 提示：你可以在左侧文件列表中找到并下载该文件。")
        
    except ImportError:
        # 兜底方案：如果环境里没有安装 openpyxl 库，就存 CSV
        print("⚠️ 检测到环境缺少 Excel 支持库 (openpyxl)，自动切换为 CSV 格式...")
        csv_filename = "HKU_Projects_Final.csv"
        df_final.to_csv(csv_filename, index=False, encoding="utf-8-sig")
        print(f"✅ 已保存为 CSV 文件: {csv_filename} (可用 Excel 打开)")
    except Exception as e:
        print(f"❌ 导出失败: {e}")

    # 8. 最后确认一下数据样子
    print("\n↓↓↓ 最终文件内容预览 ↓↓↓")
    display(df_final.head())

else:
    print("❌ 错误：内存中找不到 'full_details_data'。请确认上面的爬虫代码已经运行完毕。")

🔄 正在整理 159 条数据...
✅ 成功导出 Excel 文件！
📂 文件名: HKU_Projects_Final.xlsx
📊 包含数据: 159 行
💡 提示：你可以在左侧文件列表中找到并下载该文件。

↓↓↓ 最终文件内容预览 ↓↓↓


,学校代码,学校名称,项目名称,项目官网链接,项目申请链接,项目opendate,项目deadline,学生案例,面试问题
0,HK001,The University of Hong Kong,Master of Science in Advanced Architectural Design (January 2026),https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-advanced-architectural-design-(sep-2026)-foa&mode=0,N/A,January 2026,"Main Round: | 12:00 noon (GMT +8), November 03, 2025",,
1,HK001,The University of Hong Kong,Master of Architecture (non-HK and International Applicants),https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(non-hk-and-international-applicants)-foa&mode=0,https://sweb.hku.hk/tola/servlet/CreateUserScreen/insertForm?curr=R399&mc=&pt=F,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
2,HK001,The University of Hong Kong,Master of Science in Conservation (Conservation Planning and Management Stream),https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-conservation-foa&mode=0,https://sweb.hku.hk/tola/servlet/CreateUserScreen/insertForm?curr=R54&mc=9T024M&pt=F,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
3,HK001,The University of Hong Kong,Master of Architecture (Design),https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-architecture-(design)-foa&mode=0,https://sweb.hku.hk/tola/servlet/CreateUserScreen/insertForm?curr=R344&mc=&pt=F,August 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
4,HK001,The University of Hong Kong,Master of Science in Conservation (Design for Conservation Stream),https://portal.hku.hk/tpg-admissions/programme-details?programme=master-of-science-in-conservation-(design-for-conservation-stream)-foa&mode=0,https://sweb.hku.hk/tola/servlet/CreateUserScreen/insertForm?curr=R54&mc=9T025M&pt=F,September 2026,"Main Round: | 12:00 noon (GMT +8), January 02, 2026",,
